In [ ]:
%matplotlib inline
import networkx as nx 
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt 


T=2
S0=1

r = 1
b = 2
a = 0

p = (r-a)/(b-a)
q = (b-r)/(b-a)

def plot_tree(g):
    pos={}
    lab={}
    
    for n in g.nodes():
        pos[n]=(n[0],n[1])
        lab[n]=float(int(g.node[n]['value']*1000))/1000 # This is just to print only with 10^-2 precision
    
    elarge=g.edges(data=True)
    nx.draw_networkx_edges(g,pos,edgelist=elarge)
    nx.draw_networkx_labels(g,pos,lab,font_size=15,font_family='sans-serif')
    plt.show()
    
def graph_stock():
    S=nx.Graph()
    for i in range(0,T):
        j=-i+1
        while (j<i+2):
            S.add_edge((i,j),(i+1,j+1))
            S.add_edge((i,j),(i+1,j-1))
            j=j+2
            
    for n in S.nodes():
        
        if n[0]==0:
            S.node[n]['value']=S0
        i=n[0]
        j=n[1]
        if j>1:
            S.node[n]['value'] = S0*((1+b)**(j-1))
        elif j==1:
            S.node[n]['value'] = S0
        else:
            S.node[n]['value'] = S0*((1+a)**(1-j))
    return S

S=nx.Graph()
S =graph_stock()
plot_tree(S)

In [ ]:
def European_Call_hedge(K):
    
    T=2
    S0=1

    r = 1
    b = 2
    a = 0

    p = (r-a)/(b-a)
    q = (b-r)/(b-a)

    g = nx.Graph()
    price = nx.Graph()
    hedge = nx.Graph()
    S = nx.Graph()
    
    S = graph_stock()

    for i in range(0,T):
            j=-i+1
            while (j<i+2):
                g.add_edge((i,j),(i+1,j+1))
                g.add_edge((i,j),(i+1,j-1))
                price.add_edge((i,j),(i+1,j+1))
                price.add_edge((i,j),(i+1,j-1))
                hedge.add_edge((i,j),(i+1,j+1))
                hedge.add_edge((i,j),(i+1,j-1))
                j=j+2

    for n in S.nodes():
        if n[0]==T:
            g.node[n]['value'] = np.maximum(S.node[n]['value']-K,0)
            price.node[n]['value'] = np.maximum(S.node[n]['value']-K,0)
            hedge.node[n]['value'] = 0
    
    i=T-1
    while i>=0:
        j=-i+1
        while j<i+2:
            g.node[(i,j)]['value'] = g.node[(i+1,j+1)]['value']*p+ g.node[(i+1,j-1)]['value']*q
            price.node[(i,j)]['value'] = g.node[(i,j)]['value']*((1.0+r)**(i-T))
            j=j+2
        i=i-1
    
    i=T-1
    while i>=0:
        j=-i+1
        while j<i+2:
            hedge.node[(i,j)]['value'] = (price.node[(i+1,j+1)]['value']-price.node[(i+1,j-1)]['value'])/(b-a)/(S.node[(i,j)]['value'])
            j=j+2
        i=i-1
        
    return hedge

In [ ]:
hedge_Call = nx.Graph()

K=0.0

K = input("Enter the value of the strike K : ")

hedge_Call = European_Call_hedge(float(K))

plot_tree(S)
plot_tree(hedge_Call)